<a href="https://colab.research.google.com/github/greyhound101/hardhat/blob/main/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv('../input/notebook9cb89c35ab/predicted.csv')
df.head()

In [ ]:
df[['XMin','XMax','YMin','YMax']]=df[['XMin','XMax','YMin','YMax']].astype('int')
df.head()

In [ ]:
from tqdm import tqdm
import cv2
import numpy as np
total=[]
for i in tqdm(df['ImageID'].unique()):
    img=cv2.imread('../input/hardhat-total/hardhat/hardhat_'+str(i)+'.jpg')
    for tmp in df.loc[df['ImageID']==i].iterrows():
        tmp=tmp[1]
        conf=tmp['Conf']
        if tmp['Conf']>0:
            color=(np.mean(img[int(tmp['YMin']):int(tmp['YMax']),int(tmp['XMin']):int(tmp['XMax']),0]),np.mean(img[int(tmp['YMin']):int(tmp['YMax']),int(tmp['XMin']):int(tmp['XMax']),1]),np.mean(img[int(tmp['YMin']):int(tmp['YMax']),int(tmp['XMin']):int(tmp['XMax']),2]))
            p1,p2=(int(tmp['XMin']),int(tmp['YMin'])),(int(tmp['XMax']),int(tmp['YMax']))
            img=cv2.rectangle(img,p1,p2,color,5)
            img=cv2.putText(img,str(conf),(p1[0],p2[1] - 2),0,5/3,[225, 255, 255], thickness=5, lineType=cv2.LINE_AA)
#     total.append(img)
    cv2.imwrite(str(i)+'.jpg',img)
